In [3]:
# Install the beir PyPI package
!pip install beir

  Using cached beir-2.0.0-py3-none-any.whl
  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)
  Using cached pytrec_eval-0.5.tar.gz (15 kB)
  Preparing metadata (setup.py) ... done
  Using cached faiss_cpu-1.9.0.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
  Using cached elasticsearch-7.9.1-py2.py3-none-any.whl.metadata (8.0 kB)
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached pyarrow-18.1.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached aiohttp-3.11.11-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Usin

In [4]:
from time import time
from beir import util, LoggingHandler
from beir.retrieval import models
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES

import logging
import pathlib, os
import random

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

dataset = "mrtydi"

#### Download nfcorpus.zip dataset and unzip the dataset
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
out_dir = os.path.join(os.getcwd(), "datasets")
data_path = util.download_and_unzip(url, out_dir)
print("Dataset downloaded here: {}".format(data_path))

#### Provide the data path where nfcorpus has been downloaded and unzipped to the data loader
# data folder would contain these files: 
# (1) nfcorpus/corpus.jsonl  (format: jsonlines)
# (2) nfcorpus/queries.jsonl (format: jsonlines)
# (3) nfcorpus/qrels/test.tsv (format: tsv ("\t"))

data_path = "datasets/mrtydi/indonesian"
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="dev")

#### Dense Retrieval using SBERT (Sentence-BERT) ####
#### Provide any pretrained sentence-transformers model
#### The model was fine-tuned using cosine-similarity.
#### Complete list - https://www.sbert.net/docs/pretrained_models.html

model = DRES(models.SentenceBERT("AryoshiW/distilbert-en-id-qa"), batch_size=128)
retriever = EvaluateRetrieval(model, score_function="dot")

#### Retrieve dense results (format of results is identical to qrels)
start_time = time()
results = retriever.retrieve(corpus, queries)
end_time = time()
print("Time taken to retrieve: {:.2f} seconds".format(end_time - start_time))
#### Evaluate your retrieval using NDCG@k, MAP@K ...

logging.info("Retriever evaluation for k in: {}".format(retriever.k_values))
ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)

mrr = retriever.evaluate_custom(qrels, results, retriever.k_values, metric="mrr")
recall_cap = retriever.evaluate_custom(qrels, results, retriever.k_values, metric="r_cap")
hole = retriever.evaluate_custom(qrels, results, retriever.k_values, metric="hole")

#### Print top-k documents retrieved ####
top_k = 10
 
query_id, ranking_scores = random.choice(list(results.items()))
scores_sorted = sorted(ranking_scores.items(), key=lambda item: item[1], reverse=True)
logging.info("Query : %s\n" % queries[query_id])

for rank in range(top_k):
    doc_id = scores_sorted[rank][0]
    # Format: Rank x: ID [Title] Body
    logging.info("Rank %d: %s [%s] - %s\n" % (rank+1, doc_id, corpus[doc_id].get("title"), corpus[doc_id].get("text")))

/opt/conda/lib/python3.11/site-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


2024-12-30 08:15:34 - Downloading mrtydi.zip ...


/workspace/datasets/mrtydi.zip:   0%|          | 0.00/8.23G [00:00<?, ?iB/s]

2024-12-30 08:32:51 - Unzipping mrtydi.zip ...
Dataset downloaded here: /workspace/datasets/mrtydi
2024-12-30 08:35:00 - Loading Corpus...


  0%|          | 0/1469399 [00:00<?, ?it/s]

...

2024-12-30 08:43:58 - Encoding Batch 29/30...


Batches:   0%|          | 0/391 [00:00<?, ?it/s]

2024-12-30 08:44:04 - Encoding Batch 30/30...


Batches:   0%|          | 0/152 [00:00<?, ?it/s]

Time taken to retrieve: 521.68 seconds
2024-12-30 08:44:06 - Retriever evaluation for k in: [1, 3, 5, 10, 100, 1000]
2024-12-30 08:44:06 - For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=False`` to ignore this.
2024-12-30 08:44:07 - 

2024-12-30 08:44:07 - NDCG@1: 0.1226
2024-12-30 08:44:07 - NDCG@3: 0.1708
2024-12-30 08:44:07 - NDCG@5: 0.1842
2024-12-30 08:44:07 - NDCG@10: 0.2014
2024-12-30 08:44:07 - NDCG@100: 0.2340
2024-12-30 08:44:07 - NDCG@1000: 0.2550
2024-12-30 08:44:07 - 

2024-12-30 08:44:07 - MAP@1: 0.1226
2024-12-30 08:44:07 - MAP@3: 0.1585
2024-12-30 08:44:07 - MAP@5: 0.1658
2024-12-30 08:44:07 - MAP@10: 0.1731
2024-12-30 08:44:07 - MAP@100: 0.1789
2024-12-30 08:44:07 - MAP@1000: 0.1796
2024-12-30 08:44:07 - 

2024-12-30 08:44:07 - Recall@1: 0.1226
2024-12-30 08:44:07 - Recall@3: 0.2067
2024-12-30 08:44:07 - Recall@5: 0.2394
2024-12-30 08:44:07 - Recall@10: 0.2917
2024-12-30 08:44:07 - Recall@100: 0.4551
20